In [1]:
 !pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import torch

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# print(device)  # Should print "cuda:0", which refers to GPU 1 due to the environment setting


In [3]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [4]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="cuda:0")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [5]:
# Freezing the base model
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the last classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datasets import Dataset

In [8]:
def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to torch float to match the expected input type for BCEWithLogitsLoss
    inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]

    return inputs


In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, (tuple, list)):
        logits = logits[0]
    print(f"LOgits shape:  {logits.shape}")
    # Sigmoid is typically used for multi-label classification to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # Convert probabilities to binary predictions (0 or 1)
    predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [11]:
model.config.output_hidden_states = True

class ModifiedModel(torch.nn.Module):
    def __init__(self, original_model):
        super(ModifiedModel, self).__init__()
        self.original_model = original_model
        self.fc = torch.nn.Linear(11, 9)  # New FC layer to map 11 -> 5 labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.original_model(input_ids, attention_mask=attention_mask)
        # Extract the last hidden state from hidden states
        last_hidden_state = outputs.hidden_states[-1]  # Shape: [batch_size, seq_length, hidden_size]
        logits = outputs.logits  # Shape: [batch_size, 11]
        logits = self.fc(logits)  # Shape: [batch_size, 5]
        if labels is not None:
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, labels.float())
            return {"loss": loss, "logits": logits, "last_hidden_state": last_hidden_state}

        return {"logits": logits, "last_hidden_state": last_hidden_state}


modified_model = ModifiedModel(model)

In [12]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="no",
    save_steps=0,
    logging_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)


##Using the trained model to get predictions on the train and dev sets

pred_labels

In [13]:
df_XED = pd.read_csv('/content/drive/MyDrive/en-annotated.tsv', delimiter='\t')
# df_XED = pd.read_csv('/hdd_storage/data/riyansha/NeuralSymbolicRegressionThatScales/emotion_class/en-annotated.tsv', delimiter='\t')
df_XED

,", ...",1
0,!,"1, 4, 7"
1,... And I don't think we need to discuss the T...,"8, 1"
2,* So get up out of your bed,1
3,A confession that you hired [PERSON] ... and a...,"1, 6"
4,A dead man has one half - hour to raise his ro...,1
...,...,...
17522,Your opinion might be valuable .,8
17523,Your orders .,8
17524,Your ship's been in lots of battles .,8
17525,"Your wine , your Majesty .",8


In [14]:
# Replace all column names
df_XED.columns = ['text', 'labels']


In [15]:
df_XED['labels'] = df_XED['labels'].apply(lambda x: [int(num) for num in x.split(',')])

In [16]:
import numpy as np

num_classes = 9

def generate_one_hot(labels, num_classes):
    one_hot = np.zeros(num_classes, dtype=float)
    for label in labels:
        one_hot[label] = 1
    return one_hot


df_XED['label'] = df_XED['labels'].apply(lambda x: generate_one_hot(x, num_classes))

In [17]:
df_XED = df_XED.drop(columns=['labels'])
df_XED

,text,label
0,!,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
1,... And I don't think we need to discuss the T...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2,* So get up out of your bed,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,A confession that you hired [PERSON] ... and a...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
4,A dead man has one half - hour to raise his ro...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...
17522,Your opinion might be valuable .,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
17523,Your orders .,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
17524,Your ship's been in lots of battles .,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
17525,"Your wine , your Majesty .","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [18]:
df_XED = df_XED.rename(columns={'label': 'labels'})

In [19]:
dataset_XED = Dataset.from_pandas(df_XED)  # Use only the desired columns


print(dataset_XED)

Dataset({
    features: ['text', 'labels'],
    num_rows: 17527
})


In [20]:
encoded_dataset_XED = dataset_XED.map(preprocess_function, batched=True)
encoded_dataset_XED

Map:   0%|          | 0/17527 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 17527
})

In [21]:
print(len(encoded_dataset_XED[0]["labels"]))

9


In [22]:
encoded_dataset_XED.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [23]:
print(encoded_dataset_XED[0]["labels"])

tensor([0., 1., 0., 0., 1., 0., 0., 1., 0.])


In [24]:
dataset_XED = encoded_dataset_XED.train_test_split(test_size=0.3)
dataset_XED

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 12268
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5259
    })
})

In [25]:

# model_ = modified_model
# model_without_fc = ModifiedModel(model, use_fc=False)


In [26]:
# Initialize Trainer
trainer = Trainer(
    model=modified_model,
    args=training_args,
    train_dataset=dataset_XED["train"],
    eval_dataset=dataset_XED["test"],
    compute_metrics=compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.312400,0.285856,0.320783,0.512918,0.692978,0.434857
2,0.290100,0.282044,0.334284,0.525279,0.692227,0.440427
3,0.284500,0.279625,0.353489,0.540793,0.698970,0.453607
4,0.279300,0.277173,0.360715,0.546593,0.693042,0.460696
5,0.276000,0.276642,0.363567,0.552397,0.692798,0.467936


LOgits shape:  (5259, 9)
LOgits shape:  (5259, 9)
LOgits shape:  (5259, 9)
LOgits shape:  (5259, 9)
LOgits shape:  (5259, 9)


TrainOutput(global_step=7670, training_loss=0.288458657793147, metrics={'train_runtime': 4281.1832, 'train_samples_per_second': 14.328, 'train_steps_per_second': 1.792, 'total_flos': 0.0, 'train_loss': 0.288458657793147, 'epoch': 5.0})

In [27]:
label_XED = {0: "neutral", 1: "anger", 2: "anticipation", 3: "disgust", 4: "fear", 5: "joy", 6: "sadness", 7: "surprise", 8: "trust"}

def decode_labels(numeric_labels):
    return [label_XED[label] for label in numeric_labels]




In [28]:
import torch

# Save the complete model
torch.save(modified_model, '/content/drive/MyDrive/Roberta_modified_XED.pth')

# torch.save(modified_model.state_dict(), '/hdd_storage/data/riyansha/NeuralSymbolicRegressionThatScales/emotion_class/Roberta_modified_XED.pth')
# Save the tokenizer
tokenizer.save_pretrained('./model_tokenizer')


('./model_tokenizer/tokenizer_config.json',
 './model_tokenizer/special_tokens_map.json',
 './model_tokenizer/vocab.json',
 './model_tokenizer/merges.txt',
 './model_tokenizer/added_tokens.json',
 './model_tokenizer/tokenizer.json')

In [ ]:
# from transformers import AutoModel
# from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Load the model
# model_trained = torch.load("/content/drive/MyDrive/Roberta_modified.pth")
# model_trained = torch.load("/hdd_storage/data/riyansha/NeuralSymbolicRegressionThatScales/emotion_class/Roberta_modified.pth")
# tokenizer = AutoTokenizer.from_pretrained('./model_tokenizer')
# model_trained.eval()
